In [ ]:
import tensorflow as tf
print(tf.__version__)

import random
import numpy as np
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint 
from sklearn import metrics  
import time

import keras

import matplotlib.pyplot as plt
import helper

from keras.models import Model,Sequential
from keras.layers import Input
from keras.layers.core import Dropout,Flatten,Dense
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.metrics import confusion_matrix,classification_report
from keras.applications.densenet import DenseNet201
from keras.applications.resnet import ResNet152

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import zipfile
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

In [ ]:
zip_ref = zipfile.ZipFile('new-plant-diseases-dataset.zip', 'r') 
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
data_dir = "/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
diseases = os.listdir(train_dir)
diseases

In [ ]:
print("Total classification classes : {}".format(len(diseases)))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen_aug = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 20,
                                   horizontal_flip = True)

test_datagen_aug = ImageDataGenerator( rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 20,
                                   horizontal_flip = True)

In [ ]:
training_set_aug = train_datagen_aug.flow_from_directory(directory= train_dir,
                                               target_size=(224, 224), 
                                               batch_size=50,
                                               class_mode='categorical') 
label_map = (training_set_aug.class_indices)
print("Target Classes Mapping Dict:\n")
print(label_map)

test_set_aug = test_datagen_aug.flow_from_directory(directory= valid_dir,
                                               target_size=(224, 224), 
                                               batch_size=50,
                                               class_mode='categorical',
                                               shuffle=False)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0025, patience=2, verbose=1)

In [ ]:
from keras.applications.resnet import ResNet50
from keras.applications.densenet import DenseNet201
from tensorflow.keras.layers import Conv2D, Input, Dense, BatchNormalization, MaxPool2D, GlobalAveragePooling2D,Dropout
from tensorflow.keras import Model

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,
                         input_shape=(224,224,3),
                         weights='imagenet',
                         pooling="avg"
                     )

base_model.trainable = False
image_input = Input(shape=(224, 224, 3))

x = base_model(image_input,training = False)

x = Dense(256,activation = "relu")(x)
x = Dropout(0.2)(x)

x = Dense(128,activation = "relu")(x)
x = Dropout(0.2)(x)

image_output = Dense(38,activation="softmax")(x)

model = Model(image_input,image_output)

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
print(model.summary())

In [ ]:
hist = model.fit(training_set_aug, epochs=20, validation_data=test_set_aug, callbacks=[callback])